# Building a Basic RAG Agent with GoodMem

## Overview

This tutorial will guide you through building a complete **Retrieval-Augmented Generation (RAG)** system using GoodMem's vector memory capabilities. By the end of this guide, you'll have a functional Q&A system that can:

- 🔍 **Semantically search** through your documents
- 📝 **Generate contextual answers** using retrieved information 
- 🏗️ **Scale to handle** large document collections

### What is RAG?

RAG combines the power of **retrieval** (finding relevant information) with **generation** (creating natural language responses). This approach allows AI systems to provide accurate, context-aware answers by:

1. **Retrieving** relevant documents from a knowledge base
2. **Augmenting** the query with this context
3. **Generating** a comprehensive answer using both the query and retrieved information

### Why GoodMem for RAG?

GoodMem provides enterprise-grade vector storage with:
- **Multiple embedder support** for optimal retrieval accuracy
- **Streaming APIs** for real-time responses
- **Advanced post-processing** with reranking and summarization
- **Scalable architecture** for production workloads


## Prerequisites

Before starting, ensure you have:

- ✅ **GoodMem server running** (install with: `curl -s https://get.goodmem.ai | bash`)
- ✅ **Node.js 14+** installed
- ✅ **npm or yarn** for package management
- ✅ **API key** for your GoodMem instance
- ✅ **OpenAI API key** (for embedder and LLM in complete RAG demo)
- ✅ **Voyage AI API key** (for reranker in complete RAG demo)

## Installation & Setup

First, let's install the required packages:

In [1]:
#!javascript
// Install the GoodMem client library
// Run in your terminal: npm install @pairsystems/goodmem-client --save

console.log('📦 To install dependencies, run:');
console.log('   npm install @pairsystems/goodmem-client');
console.log('\n💡 Make sure Node.js 14+ is installed');

📦 To install dependencies, run:
   npm install @pairsystems/goodmem-client

💡 Make sure Node.js 14+ is installed


undefined

## Authentication & Configuration

### Why This Matters

GoodMem uses API key authentication to secure your vector memory data. Proper configuration ensures:
- **Secure access** to your GoodMem instance
- **Isolated environments** (development, staging, production)
- **Usage tracking** and access control per API key

### What We'll Do

1. Configure the GoodMem host URL (where your server is running)
2. Set up API key authentication
3. Verify the configuration is correct

### Configuration Options

- **Local development**: `http://localhost:8080` (default)
- **Remote/production**: Your deployed GoodMem URL
- **Environment variables**: Best practice for managing credentials

Let's configure our GoodMem client and test the connection:

In [2]:
const GoodMemClient = require('@pairsystems/goodmem-client');
const fs = require('fs').promises;
const path = require('path');

// Configuration - Update these values for your setup
const GOODMEM_HOST = process.env.GOODMEM_HOST || 'http://localhost:8080';
const GOODMEM_API_KEY = process.env.GOODMEM_API_KEY || '';

console.log('GoodMem Host:', GOODMEM_HOST);
console.log('API Key configured:', GOODMEM_API_KEY !== 'your-api-key-here' ? 'Yes' : 'No - Please update');

// Create and configure API client
const apiClient = new GoodMemClient.ApiClient();
apiClient.basePath = GOODMEM_HOST;
apiClient.defaultHeaders = {
    'X-API-Key': GOODMEM_API_KEY,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
};

// Create API instances
const spacesApi = new GoodMemClient.SpacesApi(apiClient);
const memoriesApi = new GoodMemClient.MemoriesApi(apiClient);
const embeddersApi = new GoodMemClient.EmbeddersApi(apiClient);
const rerankersApi = new GoodMemClient.RerankersApi(apiClient);
const llmsApi = new GoodMemClient.LLMsApi(apiClient);
const streamingClient = new GoodMemClient.StreamingClient(apiClient);

GoodMem Host: http://localhost:8080
API Key configured: Yes


{
  'X-API-Key': 'gm_****',
  'Content-Type': 'application/json',
  Accept: 'application/json'
}

In [3]:
// Test connection by listing existing spaces
spacesApi.listSpaces()
    .then(response => {
        console.log('✅ Successfully connected to GoodMem!');
        const spaces = response.spaces || [];
        console.log(`   Found ${spaces.length} existing spaces`);
    });

Promise { <pending> }

✅ Successfully connected to GoodMem!
   Found 0 existing spaces


## Creating an Embedder

### Why Embedders Matter

An **embedder** is the foundation of semantic search. It converts text into high-dimensional vectors (embeddings) that capture meaning:

```
Text: "vacation policy" → Vector: [0.23, -0.45, 0.67, ...]  (1536 dimensions)
```

These vectors enable:
- **Semantic similarity**: Find conceptually similar content, not just keyword matches
- **Context understanding**: Capture meaning beyond exact word matches
- **Efficient retrieval**: Fast vector comparisons using specialized indexes

### The RAG Pipeline Flow

```
Documents → Embedder → Vector Storage → Semantic Search → Retrieved Context
```

### Choosing an Embedder

**OpenAI `text-embedding-3-small`** (what we'll use):
- ✅ **High quality**: Excellent for most use cases
- ✅ **Fast**: Low latency for real-time applications  
- ✅ **1536 dimensions**: Good balance of quality and storage
- ✅ **Cost-effective**: $0.02 per 1M tokens

**Other options**:
- **text-embedding-3-large**: Higher quality, 3072 dimensions, more expensive
- **Voyage AI**: Specialized for search, excellent retrieval performance
- **Cohere**: Good multilingual support
- **Local models**: HuggingFace sentence transformers for privacy/offline

### What We'll Do

1. Check if an embedder already exists
2. If not, create an OpenAI embedder with proper authentication
3. Verify the embedder is ready for use

**Note**: You'll need an OpenAI API key set in your environment variable `OPENAI_API_KEY`.

In [4]:
// Create or retrieve OpenAI embedder
let openaiEmbedder = null;
const openaiApiKey = process.env.OPENAI_API_KEY || '';

async function createOpenaiEmbedder() {    
    if (!openaiApiKey) {
        console.log('❌ OPENAI_API_KEY environment variable not set!');
        return null;
    }

    // Create API key authentication
    const apiKeyAuth = {
        inlineSecret: openaiApiKey,
        headerName: 'Authorization',
        prefix: 'Bearer '
    };
    
    const credentials = {
        kind: 'CREDENTIAL_KIND_API_KEY',
        apiKey: apiKeyAuth
    };
    
    // Create embedder request
    const embedderRequest = {
        displayName: 'OpenAI Text Embedding 3 Small',
        providerType: 'OPENAI',
        endpointUrl: 'https://api.openai.com/v1',
        modelIdentifier: 'text-embedding-3-small',
        dimensionality: 1536,
        apiPath: '/embeddings',
        distributionType: 'DENSE',
        supportedModalities: ['TEXT'],
        credentials: credentials
    };
    
    const newEmbedder = await embeddersApi.createEmbedder(embedderRequest);
    
    console.log('✅ Successfully created OpenAI embedder!');
    console.log(`   Display Name: ${newEmbedder.displayName}`);
    console.log(`   Embedder ID: ${newEmbedder.embedderId}`);
    console.log(`   Provider: ${newEmbedder.providerType}`);
    console.log(`   Model: ${newEmbedder.modelIdentifier}`);
    console.log(`   Dimensionality: ${newEmbedder.dimensionality}`);
    
    return newEmbedder;
}

createOpenaiEmbedder()
    .then(embedder => {
        openaiEmbedder = embedder;
    });

Promise { <pending> }

✅ Successfully created OpenAI embedder!
   Display Name: OpenAI Text Embedding 3 Small
   Embedder ID: 019b33d1-6741-76ef-b1a0-a25e6aec984a
   Provider: OPENAI
   Model: text-embedding-3-small
   Dimensionality: 1536


## Creating Your First Space

### What is a Space?

A **Space** in GoodMem is a logical container for organizing related memories (documents). Think of it as a database or collection where you store and retrieve semantically similar content.

Each space has:
- **Associated embedders**: Which models convert text to vectors
- **Chunking configuration**: How documents are split into searchable pieces
- **Access controls**: Public or private, with permission management
- **Metadata labels**: For organization and filtering

### Use Cases for Multiple Spaces

You might create different spaces for:
- **By domain**: Technical docs, HR policies, product specs
- **By environment**: Development, staging, production
- **By customer**: Tenant-specific data in multi-tenant apps
- **By privacy level**: Public FAQ vs. internal knowledge base

### Why Chunking Matters

Documents are too large to search efficiently as whole units. Chunking:
- **Improves relevance**: Match specific sections, not entire documents
- **Enables context**: Return focused chunks that answer specific questions  
- **Optimizes retrieval**: Process and compare smaller text segments

**Our chunking strategy**:
- **256 characters**: Short enough for focused context, long enough for meaning
- **25 character overlap**: Ensures concepts spanning chunk boundaries aren't lost
- **Hierarchical separators**: Split on paragraphs first, then sentences, then words

### What We'll Do

1. List available embedders
2. Create a space with our embedder and chunking configuration
3. Add metadata labels for organization
4. Verify the space is ready

Let's create a space for our RAG demo:

In [5]:
// First, let's see what embedders are available
let availableEmbedders = [];
let defaultEmbedder = null;

embeddersApi.listEmbedders()
    .then(embeddersResponse => {
        availableEmbedders = embeddersResponse.embedders || [];
        
        console.log(`📋 Available Embedders (${availableEmbedders.length}):`);
        availableEmbedders.forEach((embedder, i) => {
            console.log(`   ${i + 1}. ${embedder.displayName} - ${embedder.providerType}`);
            console.log(`      Model: ${embedder.modelIdentifier || 'N/A'}`);
            console.log(`      ID: ${embedder.embedderId}`);
            console.log();
        });
        
        defaultEmbedder = availableEmbedders[0];
        console.log(`🎯 Using embedder: ${defaultEmbedder.displayName}`);
    })
    ;

Promise { <pending> }

📋 Available Embedders (1):
   1. OpenAI Text Embedding 3 Small - OPENAI
      Model: text-embedding-3-small
      ID: 019b33d1-6741-76ef-b1a0-a25e6aec984a

🎯 Using embedder: OpenAI Text Embedding 3 Small


In [6]:
// Create a space for our RAG demo
const SPACE_NAME = 'RAG Demo Knowledge Base (JavaScript)';
let demoSpace = null;

// Define chunking configuration
const DEMO_CHUNKING_CONFIG = {
    recursive: {
        chunkSize: 256,
        chunkOverlap: 25,
        separators: ['\n\n', '\n', '. ', ' ', ''],
        keepStrategy: 'KEEP_END',
        separatorIsRegex: false,
        lengthMeasurement: 'CHARACTER_COUNT'
    }
};

console.log('📋 Demo Chunking Configuration:');
console.log(`   Chunk Size: ${DEMO_CHUNKING_CONFIG.recursive.chunkSize} characters`);
console.log(`   Overlap: ${DEMO_CHUNKING_CONFIG.recursive.chunkOverlap} characters`);
console.log(`   Strategy: ${DEMO_CHUNKING_CONFIG.recursive.keepStrategy}\n`);

// Create space embedders
const spaceEmbedders = [];
if (defaultEmbedder) {
    spaceEmbedders.push({
        embedderId: defaultEmbedder.embedderId,
        defaultRetrievalWeight: 1.0
    });
}

// Create space directly
const createRequest = {
    name: SPACE_NAME,
    labels: {
        purpose: 'rag-demo',
        environment: 'tutorial',
        'content-type': 'documentation',
        language: 'javascript'
    },
    spaceEmbedders: spaceEmbedders,
    publicRead: false,
    defaultChunkingConfig: DEMO_CHUNKING_CONFIG
};

spacesApi.createSpace(createRequest)
    .then(space => {
        demoSpace = space;
        console.log(`✅ Created space: ${space.name}`);
        console.log(`   Space ID: ${space.spaceId}`);
        console.log(`   Embedders: ${space.spaceEmbedders ? space.spaceEmbedders.length : 0}`);
        console.log(`   Labels: ${JSON.stringify(space.labels)}`);
        console.log(`   Chunking Config: ${DEMO_CHUNKING_CONFIG.recursive.chunkSize} chars with ${DEMO_CHUNKING_CONFIG.recursive.chunkOverlap} overlap`);
    });

📋 Demo Chunking Configuration:
   Chunk Size: 256 characters
   Overlap: 25 characters
   Strategy: KEEP_END



Promise { <pending> }

✅ Created space: RAG Demo Knowledge Base (JavaScript)
   Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
   Embedders: 1
   Labels: {"purpose":"rag-demo","environment":"tutorial","content-type":"documentation","language":"javascript"}
   Chunking Config: 256 chars with 25 overlap


In [7]:
// Verify our space configuration
spacesApi.getSpace(demoSpace.spaceId)
    .then(spaceDetails => {
        console.log('🔍 Space Configuration:');
        console.log(`   Name: ${spaceDetails.name}`);
        console.log(`   Owner ID: ${spaceDetails.ownerId}`);
        console.log(`   Public Read: ${spaceDetails.publicRead}`);
        console.log(`   Created: ${new Date(spaceDetails.createdAt).toISOString()}`);
        console.log(`   Labels: ${JSON.stringify(spaceDetails.labels)}`);
        
        console.log('\n🤖 Associated Embedders:');
        if (spaceDetails.spaceEmbedders && spaceDetails.spaceEmbedders.length > 0) {
            spaceDetails.spaceEmbedders.forEach(embedderAssoc => {
                console.log(`   Embedder ID: ${embedderAssoc.embedderId}`);
                console.log(`   Retrieval Weight: ${embedderAssoc.defaultRetrievalWeight}`);
            });
        }
    });

Promise { <pending> }

🔍 Space Configuration:
   Name: RAG Demo Knowledge Base (JavaScript)
   Owner ID: cf5df949-31c6-4c54-af50-f8002107164e
   Public Read: false
   Created: 2025-12-18T23:37:29.159Z
   Labels: {"purpose":"rag-demo","language":"javascript","environment":"tutorial","content-type":"documentation"}

🤖 Associated Embedders:
   Embedder ID: 019b33d1-6741-76ef-b1a0-a25e6aec984a
   Retrieval Weight: 1


## Adding Documents to Memory

### The Document Processing Pipeline

When you add a document to GoodMem, it goes through several automated steps:

```
1. Ingestion → 2. Chunking → 3. Embedding → 4. Indexing → 5. Ready for Search
```

**What happens**:
1. **Ingestion**: Document content and metadata are stored
2. **Chunking**: Text is split according to your configuration (256 chars, 25 overlap)
3. **Embedding**: Each chunk is converted to a vector by your embedder
4. **Indexing**: Vectors are indexed for fast similarity search
5. **Status**: Document marked as `COMPLETED` and ready for retrieval

### Single vs. Batch Operations

**Single memory creation** (`CreateMemory`):
- ✅ Good for: Real-time ingestion, single documents
- ✅ Synchronous processing with immediate status
- ⚠️ Higher overhead for bulk operations

**Batch memory creation** (`BatchCreateMemory`):
- ✅ Good for: Bulk imports, initial setup, periodic updates
- ✅ Lower overhead, efficient for multiple documents
- ✅ Async processing - check status via `ListMemories`
- ⚠️ Takes longer to get individual status feedback

### Metadata Best Practices

Rich metadata helps with:
- **Filtering**: Retrieve specific document types
- **Source attribution**: Show users where information came from
- **Organization**: Group and manage related documents
- **Debugging**: Track ingestion methods and dates

### What We'll Do

1. Load sample documents from local files
2. Create one document using single memory creation (to demo the API)
3. Create remaining documents using batch operation (more efficient)
4. Monitor processing status until all documents are ready

We'll use sample company documents that represent common business use cases:

In [8]:
// Load sample documents
async function loadSampleDocuments() {
    const documents = [];
    const sampleDir = 'sample_documents';
    
    // Check if directory exists
    await fs.access(sampleDir);
    
    // Read all files in directory
    const files = await fs.readdir(sampleDir);
    
    // Sort files for consistent ordering
    files.sort();
    
    for (const filename of files) {
        const filepath = path.join(sampleDir, filename);
        const ext = path.extname(filename).toLowerCase();
        
        // Check if it's a file (not a directory)
        const stats = await fs.stat(filepath);
        if (!stats.isFile()) continue;
        
        if (ext === '.txt') {
            // Handle text files
            const content = await fs.readFile(filepath, 'utf-8');
            documents.push({
                filename,
                content,
                contentType: 'text/plain',
                isBinary: false
            });
            console.log(`📄 Loaded: ${filename} (${content.length.toLocaleString()} characters)`);
        } else if (ext === '.pdf') {
            // Handle PDF files with base64 encoding
            const binaryContent = await fs.readFile(filepath);
            const contentB64 = binaryContent.toString('base64');
            documents.push({
                filename,
                contentB64,
                contentType: 'application/pdf',
                isBinary: true
            });
            console.log(`📄 Loaded: ${filename} (${binaryContent.length.toLocaleString()} bytes, base64: ${contentB64.length.toLocaleString()} chars)`);
        } else {
            console.log(`⚠️  Skipping unsupported file type: ${filename}`);
        }
    }
    
    return documents;
}

// Load the documents
let sampleDocs = [];
loadSampleDocuments()
    .then(docs => {
        sampleDocs = docs;
        console.log(`\n📚 Total documents loaded: ${docs.length}`);
    });

Promise { <pending> }

📄 Loaded: company_handbook.txt (2,342 characters)
📄 Loaded: employee_handbook.pdf (399,615 bytes, base64: 532,820 chars)
📄 Loaded: product_faq.txt (4,043 characters)
📄 Loaded: security_policy.txt (4,211 characters)
📄 Loaded: technical_documentation.txt (2,384 characters)

📚 Total documents loaded: 5


In [9]:
// Helper function to build memory request
function buildMemoryRequest(spaceId, document) {
    // Build a memory creation request based on document type.
    const memoryRequest = {
        spaceId: spaceId,
        contentType: document.contentType,
        chunkingConfig: DEMO_CHUNKING_CONFIG,
        metadata: {
            filename: document.filename,
            source: 'sample_documents',
        }
    };
    
    // Add content field based on type
    if (document.isBinary) {
        memoryRequest.originalContentB64 = document.contentB64;  // Base64 for PDFs
    } else {
        memoryRequest.originalContent = document.content;         // Plain text
    }
    
    return memoryRequest;
}

// Create the first memory individually
let singleMemory = null;
const firstDoc = sampleDocs[0];
console.log('📝 Creating first document using CreateMemory API:');
console.log(`   Document: ${firstDoc.filename}`);
console.log(`   Content Type: ${firstDoc.contentType}`);

const memoryRequest = buildMemoryRequest(demoSpace.spaceId, firstDoc);
memoriesApi.createMemory(memoryRequest)
    .then(memory => {
        singleMemory = memory;
        console.log(`✅ Created single memory: ${firstDoc.filename}`);
        console.log(`   Memory ID: ${memory.memoryId}`);
        console.log(`   Content Type: ${memory.contentType}`);
        console.log(`   Status: ${memory.processingStatus}`);
        console.log();
        console.log('🎯 Single memory creation completed successfully!');
    });

📝 Creating first document using CreateMemory API:
   Document: company_handbook.txt
   Content Type: text/plain



Promise { <pending> }

✅ Created single memory: company_handbook.txt
   Memory ID: 019b33d4-7e4a-7002-a9d9-ef3d6e0d2b59
   Content Type: text/plain
   Status: PENDING

🎯 Single memory creation completed successfully!


In [10]:
// Retrieve memory by ID
memoriesApi.getMemory(singleMemory.memoryId, { includeContent: true })
    .then(retrievedMemory => {
        console.log('✅ Successfully retrieved memory:');
        console.log(`   Memory ID: ${retrievedMemory.memoryId}`);
        console.log(`   Space ID: ${retrievedMemory.spaceId}`);
        console.log(`   Status: ${retrievedMemory.processingStatus}`);
        console.log(`   Content Type: ${retrievedMemory.contentType}`);
        console.log(`   Created At: ${new Date(retrievedMemory.createdAt).toISOString()}`);
        console.log(`   Updated At: ${new Date(retrievedMemory.updatedAt).toISOString()}`);
        
        if (retrievedMemory.metadata) {
            console.log('\n   📋 Metadata:');
            for (const [key, value] of Object.entries(retrievedMemory.metadata)) {
                console.log(`      ${key}: ${value}`);
            }
        }
        
        if (retrievedMemory.originalContent) {
            // Decode the base64 encoded content
            const decodedContent = Buffer.from(retrievedMemory.originalContent, 'base64').toString('utf-8');
            
            console.log('✅ Content retrieved and decoded:');
            console.log(`   Content length: ${decodedContent.length} characters`);
            const preview = decodedContent.length > 200 ? decodedContent.substring(0, 200) + '...' : decodedContent;
            console.log(`   First 200 chars: ${preview}`);
        }
    })

Promise { <pending> }

✅ Successfully retrieved memory:
   Memory ID: 019b33d4-7e4a-7002-a9d9-ef3d6e0d2b59
   Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
   Status: COMPLETED
   Content Type: text/plain
   Created At: 2025-12-18T23:38:42.635Z
   Updated At: 2025-12-18T23:38:45.566Z

   📋 Metadata:
      source: sample_documents
      filename: company_handbook.txt
✅ Content retrieved and decoded:
   Content length: 2342 characters
   First 200 chars: ACME Corporation Employee Handbook

Welcome to ACME Corporation! This handbook provides essential information about our company policies, procedures, and culture.

COMPANY OVERVIEW
ACME Corporation is...


In [11]:
// Create remaining documents using batch creation
let createdMemories = [];
const remainingDocs = sampleDocs.slice(1);

// Build memory requests using our helper function
const memoryRequests = remainingDocs.map(doc => 
    buildMemoryRequest(demoSpace.spaceId, doc)
);

const batchRequest = {
    requests: memoryRequests
};

memoriesApi.batchCreateMemory(batchRequest)
    .then(response => {
        // Save the created memories from the response
        createdMemories = response.results.map(item => item.memory);
        
        console.log('✅ Batch creation request submitted successfully');
        console.log('\n📋 Total Memory Creation Summary:');
        console.log('   📄 Single CreateMemory: 1 document');
        console.log(`   📦 Batch CreateMemory: ${createdMemories.length} documents created`);
        console.log('   ⏳ Check processing status in the next cell');
    });

Promise { <pending> }

✅ Batch creation request submitted successfully

📋 Total Memory Creation Summary:
   📄 Single CreateMemory: 1 document
   📦 Batch CreateMemory: 4 documents created
   ⏳ Check processing status in the next cell


In [12]:
// Batch retrieve memories to check their status
async function batchRetrieveMemories(memoryIds) {
    // Retrieve multiple memories by their IDs using batch retrieval.
    const batchRequest = {
        memoryIds: memoryIds,
        includeContent: false  // We don't need content for status check
    };
    
    const response = await memoriesApi.batchGetMemory(batchRequest);
    return response.results.map(item => item.memory);
}

// Get memory IDs from single + batch creation
const memoryIds = [singleMemory.memoryId, ...createdMemories.map(m => m.memoryId)];

batchRetrieveMemories(memoryIds)
    .then(memories => {
        console.log(`📚 Retrieved ${memories.length} memories:\n`);
        
        memories.forEach((memory, i) => {
            const metadata = memory.metadata || {};
            const filename = metadata.filename || 'Unknown';
            const source = metadata.source || 'Unknown';
            
            console.log(`   ${i + 1}. ${filename}`);
            console.log(`      Status: ${memory.processingStatus}`);
            console.log(`      Source: ${source}`);
            console.log(`      Created: ${new Date(memory.createdAt).toISOString()}\n`);
        });
    });

Promise { <pending> }

📚 Retrieved 5 memories:

   1. company_handbook.txt
      Status: COMPLETED
      Source: sample_documents
      Created: 2025-12-18T23:38:42.635Z

   2. employee_handbook.pdf
      Status: COMPLETED
      Source: sample_documents
      Created: 2025-12-18T23:40:59.163Z

   3. product_faq.txt
      Status: COMPLETED
      Source: sample_documents
      Created: 2025-12-18T23:40:59.163Z

   4. security_policy.txt
      Status: COMPLETED
      Source: sample_documents
      Created: 2025-12-18T23:40:59.163Z

   5. technical_documentation.txt
      Status: COMPLETED
      Source: sample_documents
      Created: 2025-12-18T23:40:59.163Z



In [13]:
// Monitor processing status using batch retrieval
async function waitForProcessingCompletion(memoryIds, maxWaitSeconds = 120) {
    console.log('⏳ Waiting for document processing to complete...');
    console.log('   💡 Note: Batch memories are processed asynchronously');
    console.log();
    
    const startTime = Date.now();
    const maxWaitMs = maxWaitSeconds * 1000;
    
    while (Date.now() - startTime < maxWaitMs) {
        const memories = await batchRetrieveMemories(memoryIds);
        
        // Check processing status
        const statusCounts = {};
        memories.forEach(memory => {
            const status = memory.processingStatus;
            statusCounts[status] = (statusCounts[status] || 0) + 1;
        });
        
        console.log(`📊 Processing status: ${JSON.stringify(statusCounts)} (Total: ${memories.length} memories)`);
        
        // Check if all are completed
        const allCompleted = memories.every(memory => memory.processingStatus === 'COMPLETED');
        if (allCompleted) {
            console.log('✅ All documents processed successfully!');
            return true;
        }
        
        // Check for any failures
        const failedCount = statusCounts['FAILED'] || 0;
        if (failedCount > 0) {
            console.log(`❌ ${failedCount} memories failed processing`);
            return false;
        }
        
        await new Promise(resolve => setTimeout(resolve, 5000));
    }
    
    console.log(`⏰ Timeout waiting for processing (waited ${maxWaitSeconds}s)`);
    return false;
}

waitForProcessingCompletion(memoryIds)
    .then(processingComplete => {
        if (processingComplete) {
            console.log('🎉 Ready for semantic search and retrieval!');
            console.log('📈 Batch API benefit: Multiple documents submitted in a single API call');
        } else {
            console.log('⚠️  Some documents may still be processing. You can continue with the tutorial.');
        }
    });

⏳ Waiting for document processing to complete...
   💡 Note: Batch memories are processed asynchronously



Promise { <pending> }

📊 Processing status: {"COMPLETED":5} (Total: 5 memories)
✅ All documents processed successfully!
🎉 Ready for semantic search and retrieval!
📈 Batch API benefit: Multiple documents submitted in a single API call


## Semantic Search & Retrieval

### Why Semantic Search?

**Traditional keyword search**:
- Matches exact words or simple variations
- Misses conceptually similar content with different wording
- Example: "vacation days" won't match "time off policy"

**Semantic search**:
- Understands meaning and context
- Finds conceptually similar content regardless of exact wording
- Example: "vacation days" successfully matches "time off policy"

### How It Works

```
Query: "vacation policy" 
   ↓ (embed with same embedder)
Query Vector: [0.23, -0.45, ...]
   ↓ (compare to all chunk vectors)
Most Similar Chunks: (by cosine similarity)
   1. "TIME OFF POLICY..." (score: -0.604)
   2. "Vacation requests..." (score: -0.544)
   3. "WORK HOURS..." (score: -0.458)
```

### Understanding Relevance Scores

GoodMem uses **cosine distance** (negative cosine similarity):
- **Lower values = more relevant** (e.g., -0.6 is better than -0.4)
- **Range**: Typically -1.0 (most similar) to 0.0 (unrelated)
- **Good threshold**: Results under -0.3 are usually relevant
- **Context matters**: Exact scores vary by embedder and content

### Streaming API Benefits

GoodMem's streaming API:
- **Real-time results**: Process chunks as they arrive
- **Low latency**: Start showing results immediately
- **Memory efficient**: No need to buffer entire result set
- **Progressive UI**: Update interface as more results come in

### What We'll Do

1. Implement a semantic search function using GoodMem's streaming API
2. Process different event types (chunks, memories, metadata)
3. Display results with relevance scores
4. Test with various queries to see semantic matching in action

Now comes the exciting part! Let's perform semantic search using GoodMem's streaming API. This will:

- **Find relevant chunks** based on semantic similarity
- **Stream results** in real-time
- **Include relevance scores** for ranking
- **Return structured data** for easy processing

In [14]:
// Perform semantic search using GoodMem's streaming API
async function semanticSearchStreaming(query, spaceId, maxResults = 5) {
    console.log(`🔍 Streaming search for: '${query}'`);
    console.log(`📁 Space ID: ${spaceId}`);
    console.log(`📊 Max results: ${maxResults}`);
    console.log('-'.repeat(50));
    
    const controller = new AbortController();
    const timeoutId = setTimeout(() => {
        console.log('⏰ Streaming timeout after 30 seconds');
        controller.abort();
    }, 30000);
    
    // Create request object
    const request = {
        message: query,
        spaceIds: [spaceId],
        requestedSize: maxResults,
        fetchMemory: true,
        fetchMemoryContent: false,
        format: GoodMemClient.StreamingClient.StreamingFormat.NDJSON
    };
    
    const stream = await streamingClient.retrieveMemoryStream(
        controller.signal,
        request
    );
    
    const retrievedChunks = [];
    let eventCount = 0;
    
    for await (const event of stream) {
        eventCount++;
        
        if (event.retrievedItem && event.retrievedItem.chunk) {
            const chunkRef = event.retrievedItem.chunk;
            const chunkData = chunkRef.chunk || {};
            const chunkText = chunkData.chunkText || '';
            
            retrievedChunks.push({
                chunkText: chunkText,
                relevanceScore: chunkRef.relevanceScore,
                memoryIndex: chunkRef.memoryIndex,
                resultSetId: chunkRef.resultSetId,
                chunkSequence: chunkData.chunkSequenceNumber || 0
            });
            
            console.log(`📄 Chunk ${retrievedChunks.length}:`);
            console.log(`   Relevance: ${chunkRef.relevanceScore.toFixed(3)}`);
            const preview = chunkText.length > 100 ? chunkText.substring(0, 100) + '...' : chunkText;
            console.log(`   Text: ${preview}`);
            console.log();
        } else if (event.resultSetBoundary) {
            console.log(`🔄 ${event.resultSetBoundary.kind}: ${event.resultSetBoundary.stageName}`);
        }
    }
    
    clearTimeout(timeoutId);
    
    console.log(`✅ Streaming search completed: ${retrievedChunks.length} chunks found, ${eventCount} events processed`);
    return retrievedChunks;
}

const sampleQuery = 'What is the vacation policy for employees?';
semanticSearchStreaming(sampleQuery, demoSpace.spaceId)
    .then(searchResults => {
        console.log(`\n✅ Search completed with ${searchResults.length} results`);
    });

🔍 Streaming search for: 'What is the vacation policy for employees?'
📁 Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
📊 Max results: 5
--------------------------------------------------


Promise { <pending> }

🔄 BEGIN: retrieve
📄 Chunk 1:
   Relevance: -0.679
   Text: TIME OFF POLICY
All full-time employees receive:
- 15 days of paid vacation annually (increases to 2...

📄 Chunk 2:
   Relevance: -0.675
   Text: 1.  Eligibility 

 
All regular full-time employees are eligible for vacation benefits. 

 
2.  Accr...

📄 Chunk 3:
   Relevance: -0.662
   Text: [ORGANIZATION] has established the following vacation plan to provide eligible employees 
time off w...

📄 Chunk 4:
   Relevance: -0.646
   Text: Vacation Pay: Vacation pay shall be based on the employee’s regular base rate and 
working schedule,...

📄 Chunk 5:
   Relevance: -0.643
   Text: employees can use paid vacation time in minimum increments of one day.xii 

 
Accumulating Vacation:...

🔄 END: 
✅ Streaming search completed: 5 chunks found, 10 events processed

✅ Search completed with 5 results


In [15]:
// Let's try a few different queries to see how streaming semantic search works
async function testMultipleStreamingQueries(spaceId) {
    const testQueries = [
        'How do I reset my password?',
        'What are the security requirements for remote work?',
        'API authentication and rate limits',
        'Employee benefits and health insurance',
        'How much does the software cost?'
    ];
    
    for (let i = 0; i < testQueries.length; i++) {
        const query = testQueries[i];
        console.log(`\n🔍 Test Query ${i + 1}: ${query}`);
        console.log('='.repeat(60));
        
        await semanticSearchStreaming(query, spaceId, 3);
        
        console.log('\n' + '-'.repeat(60));
    }
}

testMultipleStreamingQueries(demoSpace.spaceId)
    .then(() => console.log('\n✅ All queries completed'));


🔍 Test Query 1: How do I reset my password?
🔍 Streaming search for: 'How do I reset my password?'
📁 Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
📊 Max results: 3
--------------------------------------------------


Promise { <pending> }

🔄 BEGIN: retrieve
📄 Chunk 1:
   Relevance: -0.370
   Text: password they use to gain access to computers or the Internet, as well as any change to 
such passwo...

📄 Chunk 2:
   Relevance: -0.363
   Text: - No reuse of last 12 passwords
- Must be changed every 90 days for privileged accounts
- Multi-fact...

📄 Chunk 3:
   Relevance: -0.305
   Text: Each classification level has specific handling, storage, and transmission requirements outlined in ...

🔄 END: 
✅ Streaming search completed: 3 chunks found, 8 events processed

------------------------------------------------------------

🔍 Test Query 2: What are the security requirements for remote work?
🔍 Streaming search for: 'What are the security requirements for remote work?'
📁 Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
📊 Max results: 3
--------------------------------------------------
🔄 BEGIN: retrieve
📄 Chunk 1:
   Relevance: -0.710
   Text: - Report suspicious emails or security incidents immediately

REMOTE WORK SECURITY
Rem

## Advanced Features

Congratulations! 🎉 You've successfully built a semantic search system using GoodMem. Here's what you've accomplished:

### ✅ What You Built
- **Document ingestion pipeline** with automatic chunking and embedding
- **Semantic search system** with relevance scoring
- **Simple Q&A system** using GoodMem's vector capabilities

### 🚀 Next Steps for Advanced Implementation

#### Reranking
Improve search quality by adding a reranking stage. **Rerankers** are specialized models that re-score search results to improve relevance:

- **Two-stage retrieval**: Fast initial retrieval with embeddings, then precise reranking
- **Better relevance**: Rerankers use cross-attention to understand query-document relationships
- **Reduced costs**: Rerank only top-K results instead of entire corpus
- **Voyage AI reranker**: Industry-leading reranking model with state-of-the-art performance

The combination of fast embedding-based retrieval followed by accurate reranking provides the best balance of speed and quality for production RAG systems.

## Configuring a Reranker

To further improve search quality, we can add a **reranker** to our RAG pipeline. While embedders provide fast semantic search, rerankers use more sophisticated models to re-score the top results for better accuracy.

### Why Use Reranking?

1. **Higher Accuracy**: Rerankers use cross-encoder architectures that directly compare queries and documents
2. **Two-Stage Pipeline**: Fast retrieval with embeddings + precise reranking = optimal performance
3. **Cost Effective**: Only rerank top-K results (e.g., top 20) rather than entire corpus

### Voyage AI Reranker

We'll use Voyage AI's `rerank-2.5` model, which provides:
- **State-of-the-art performance** on reranking benchmarks
- **Fast inference** optimized for production use
- **Simple API** that integrates seamlessly with GoodMem

**Note**: You'll need a Voyage AI API key set in your environment variable `VOYAGE_API_KEY`.

In [16]:
// Create or retrieve Voyage AI reranker
let voyageReranker = null;
const voyageApiKey = process.env.VOYAGE_API_KEY || 'pa-38AtcNwuEpQMYof5sdEHwLitnKLKC-vbMAwMPmjOT6D';

async function createVoyageReranker() {
    if (!voyageApiKey) {
        console.log('❌ VOYAGE_API_KEY environment variable not set!');
        return null;
    }
    
    // Create API key authentication
    const apiKeyAuth = {
        inlineSecret: voyageApiKey,
        headerName: 'Authorization',
        prefix: 'Bearer '
    };
    
    const credentials = {
        kind: 'CREDENTIAL_KIND_API_KEY',
        apiKey: apiKeyAuth
    };
    
    // Create reranker request
    const rerankerRequest = {
        displayName: 'Voyage Rerank 2.5',
        providerType: 'VOYAGE',
        endpointUrl: 'https://api.voyageai.com',
        modelIdentifier: 'rerank-2.5',
        apiPath: '/v1/rerank',
        credentials: credentials,
        description: 'Voyage AI reranker for improving search result relevance'
    };
    
    const newReranker = await rerankersApi.createReranker(rerankerRequest);
    
    console.log('✅ Successfully created Voyage reranker!');
    console.log(`   Display Name: ${newReranker.displayName}`);
    console.log(`   Reranker ID: ${newReranker.rerankerId}`);
    console.log(`   Provider: ${newReranker.providerType}`);
    console.log(`   Model: ${newReranker.modelIdentifier}`);
    
    return newReranker;
}

createVoyageReranker()
    .then(reranker => {
        voyageReranker = reranker;
    });

Promise { <pending> }

✅ Successfully created Voyage reranker!
   Display Name: Voyage Rerank 2.5
   Reranker ID: 019b33d9-5317-70ff-abd4-270f64aa0a51
   Provider: VOYAGE
   Model: rerank-2.5


## Registering an LLM

The final component in our RAG pipeline is the **LLM (Large Language Model)** - the generation component that creates natural language responses using the retrieved and reranked context.

### Role of LLMs in RAG

After retrieving and reranking relevant chunks, the LLM:
1. **Receives the query** and retrieved context
2. **Generates a response** that synthesizes information from multiple sources
3. **Maintains coherence** while staying grounded in the retrieved facts

### OpenAI GPT-4o-mini

We'll use OpenAI's `gpt-4o-mini` model, which provides:
- **Fast inference** with low latency for real-time applications
- **Cost-effective** pricing compared to larger models
- **High quality** responses suitable for most RAG use cases
- **Function calling** support for advanced workflows

**Note**: This uses the same `OPENAI_API_KEY` environment variable as the embedder.

In [17]:
// Register OpenAI GPT-4o-mini LLM
let openaiLlm = null;

async function createOpenaiLlm() {
    if (!openaiApiKey) {
        console.log('❌ OPENAI_API_KEY environment variable not set!');
        return null;
    }
    
    // Create API key authentication
    const apiKeyAuth = {
        inlineSecret: openaiApiKey,
        headerName: 'Authorization',
        prefix: 'Bearer '
    };
    
    const credentials = {
        kind: 'CREDENTIAL_KIND_API_KEY',
        apiKey: apiKeyAuth
    };
    
    // Define LLM capabilities
    const capabilities = {
        supportsChat: true,
        supportsCompletion: false,
        supportsFunctionCalling: true,
        supportsSystemMessages: true,
        supportsStreaming: true,
        supportsSamplingParameters: true
    };
    
    // Create LLM request
    const llmRequest = {
        displayName: 'OpenAI GPT-4o Mini',
        providerType: 'OPENAI',
        endpointUrl: 'https://api.openai.com/v1',
        modelIdentifier: 'gpt-4o-mini',
        apiPath: '/chat/completions',
        credentials: credentials,
        capabilities: capabilities,
        description: "OpenAI's GPT-4o Mini model for fast and efficient text generation"
    };
    
    const response = await llmsApi.createLLM(llmRequest);
    const newLLM = response.llm || response;
    
    console.log('✅ Successfully registered OpenAI GPT-4o-mini LLM!');
    console.log(`   Display Name: ${newLLM.displayName}`);
    console.log(`   LLM ID: ${newLLM.llmId}`);
    console.log(`   Provider: ${newLLM.providerType}`);
    console.log(`   Model: ${newLLM.modelIdentifier}`);
    
    return newLLM;
}

createOpenaiLlm()
    .then(llm => {
        openaiLlm = llm;
    });

Promise { <pending> }

✅ Successfully registered OpenAI GPT-4o-mini LLM!
   Display Name: OpenAI GPT-4o Mini
   LLM ID: 019b33da-023b-7095-9702-76a149e6e578
   Provider: OPENAI
   Model: gpt-4o-mini


## Enhanced RAG with Reranking and LLM Generation

Now that we have all the components configured (embedder, reranker, and LLM), let's use the complete RAG pipeline! This demonstrates the full power of GoodMem:

1. **Retrieval**: Fast semantic search finds relevant chunks
2. **Reranking**: Voyage AI reranker re-scores results for better relevance  
3. **Generation**: OpenAI GPT-4o-mini generates a coherent response using the reranked context

This provides significantly better answer quality compared to simple retrieval alone.

In [18]:
// Helper classes for RAG results
class RagChunk {
    constructor(chunkText, relevanceScore) {
        this.chunkText = chunkText;
        this.relevanceScore = relevanceScore;
    }
}

class RagResult {
    constructor(llmResponse, chunks) {
        this.llmResponse = llmResponse;
        this.chunks = chunks;
    }
}

// RAG pipeline function
async function ragPipelineStreaming(query, spaceId, rerankerId, llmId, firstStageSize, maxResults) {
    console.log(`🔍 RAG Query: '${query}'`);
    console.log(`📁 Space ID: ${spaceId}`);
    console.log(`📊 Max results: ${maxResults}`);
    console.log('='.repeat(70));

    // Build post-processor configuration
    const postProcessorConfig = {
        llm_id: llmId,
        reranker_id: rerankerId,
        relevance_threshold: 0.3,
        max_results: maxResults
    };

    // Create memory stream request with post-processor
    const request = {
        message: query,
        spaceIds: [spaceId],
        requestedSize: firstStageSize,
        fetchMemory: true,
        fetchMemoryContent: false,
        postProcessorName: 'com.goodmem.retrieval.postprocess.ChatPostProcessorFactory',
        postProcessorConfig: postProcessorConfig,
        format: GoodMemClient.StreamingClient.StreamingFormat.NDJSON
    };

    let llmResponse = null;
    const rerankedChunks = [];

    const controller = new AbortController();
    const timeoutId = setTimeout(() => controller.abort(), 60000);

    // Process streaming events
    const stream = await streamingClient.retrieveMemoryStreamAdvanced(
        controller.signal,
        request
    );

    for await (const streamingEvent of stream) {
        // Handle LLM-generated response
        if (streamingEvent.abstractReply && !llmResponse) {
            llmResponse = streamingEvent.abstractReply.text;
            console.log('\n🤖 LLM Generated Response:');
            console.log(`   ${llmResponse}`);
            console.log();
            console.log('-'.repeat(70));
        }

        // Handle reranked chunks
        if (streamingEvent.retrievedItem && streamingEvent.retrievedItem.chunk) {
            const chunkRef = streamingEvent.retrievedItem.chunk;
            const chunkData = chunkRef.chunk || {};

            const chunkText = chunkData.chunkText || '';
            const relevanceScore = chunkRef.relevanceScore;
            rerankedChunks.push(new RagChunk(chunkText, relevanceScore));

            console.log(`   📄 Chunk ${rerankedChunks.length}:`);
            console.log(`      Relevance: ${relevanceScore.toFixed(3)}`);
            const preview = chunkText.length > 150 ? chunkText.substring(0, 150) + '...' : chunkText;
            console.log(`      Text: ${preview}`);
            console.log();
        }
    }

    clearTimeout(timeoutId);
    return new RagResult(llmResponse, rerankedChunks);
}

// Usage
const testQuery = 'What is the vacation policy for employees?';

ragPipelineStreaming(
    testQuery,
    demoSpace.spaceId,
    voyageReranker.rerankerId,
    openaiLlm.llmId,
    10,
    3
).then(ragResult => {
    if (ragResult) {
        console.log('\n🎉 RAG pipeline completed!');
        console.log(`   LLM Response: ${ragResult.llmResponse ? 'Available' : 'None'}`);
        console.log(`   Chunks Retrieved: ${ragResult.chunks.length}`);
    }
});

🔍 RAG Query: 'What is the vacation policy for employees?'
📁 Space ID: 019b33d3-5f46-7454-945a-ae7ea4c85475
📊 Max results: 3


Promise { <pending> }

   📄 Chunk 1:
      Relevance: 0.863
      Text: TIME OFF POLICY
All full-time employees receive:
- 15 days of paid vacation annually (increases to 20 days after 3 years)
- 10 sick days per year
- 8 ...

   📄 Chunk 2:
      Relevance: 0.824
      Text: [ORGANIZATION] has established the following vacation plan to provide eligible employees 
time off with pay so that they may be free from their regula...

   📄 Chunk 3:
      Relevance: 0.777
      Text: employees can use paid vacation time in minimum increments of one day.xii 

 
Accumulating Vacation: Employees are encouraged to use available paid va...


🤖 LLM Generated Response:
   The vacation policy for employees includes 15 days of paid vacation annually, which increases to 20 days after three years of employment. Additionally, employees receive 10 sick days per year, 8 company holidays, and personal days as needed with manager approval. Vacation time can be used in minimum increments of one day, and employees are encouraged to ut

## 🎉 Congratulations! What You Built

You've successfully built a complete **Retrieval-Augmented Generation (RAG) system** using GoodMem! Let's recap what you accomplished.

### Components You Configured

| Component | Purpose | Function |
|-----------|---------|----------|
| **Embedder** | Convert text to vectors | Transform documents into semantic embeddings |
| **Space** | Organize and store documents | Logical container with chunking configuration |
| **Memories** | Store searchable content | Documents chunked and indexed for retrieval |
| **Reranker** | Improve search precision | Re-score results for better relevance |
| **LLM** | Generate natural language | Create coherent answers from retrieved context |

### The Complete RAG Pipeline

```
📄 Documents
   ↓ Chunking (256 chars, 25 overlap)
   ↓ Embedding (convert to vectors)
🗄️  Vector Storage (GoodMem Space)
   ↓ 
🔍 User Query
   ↓ Semantic Search (retrieve top-K)
   ↓ Reranking (re-score for precision)
   ↓ Context Selection (most relevant chunks)
🤖 LLM Generation (synthesize answer)
   ↓
✨ Natural Language Answer
```

### Key Concepts You Learned

1. **Embedders**: Transform text into semantic vectors for similarity search
2. **Spaces**: Logical containers for organizing and searching documents
3. **Chunking**: Breaking documents into optimal sizes for retrieval
4. **Semantic Search**: Finding conceptually similar content, not just keyword matches
5. **Reranking**: Two-stage retrieval for better precision
6. **Streaming API**: Real-time, memory-efficient result processing
7. **RAG Architecture**: Combining retrieval and generation for accurate, grounded responses

### Performance Improvements

**Basic search** (retrieval only):
- Fast retrieval using vector similarity
- Good recall, but may include less relevant results

**Enhanced RAG** (with reranker + LLM):
- Reranker improves precision significantly
- LLM synthesizes information from multiple chunks
- Better user experience with natural language answers
- Grounded in actual document content (no hallucinations)

### Next Steps & Advanced Topics

**Enhance Your RAG System**:
- **Multiple embedders**: Combine different embedders for better coverage
- **Custom chunking**: Tune chunk size/overlap for your content type
- **Metadata filtering**: Add filters to narrow search by document type, date, etc.
- **Hybrid search**: Combine semantic and keyword search
- **Context augmentation**: Include surrounding chunks for better LLM context

**Production Deployment**:
- **Monitoring**: Track query latency, relevance scores, user feedback
- **Scaling**: Horizontal scaling for high-traffic applications
- **Cost optimization**: Balance quality vs. API costs
- **Caching**: Cache frequent queries for faster responses
- **Error handling**: Robust exception handling and retry logic

**Advanced Features**:
- **Multi-space search**: Query across multiple knowledge bases
- **Query expansion**: Transform queries for better retrieval
- **Result aggregation**: Combine and deduplicate results
- **Streaming generation**: Progressive LLM responses for real-time UX
- **Fine-tuning**: Customize models for your specific domain

### Resources

- **Documentation**: [https://docs.goodmem.ai](https://docs.goodmem.ai)
- **Community**: Join discussions and share your implementations
- **Examples**: Explore more advanced use cases and patterns

---

**Great job!** You now have a solid foundation for building production RAG systems with GoodMem. 🚀
